# コードのテスト
## define_model.py 関連の関数について
- ニューラルネットワーク構築する

In [8]:
import os
gpu_num = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_num)

# モジュールimport
import os, sys
current_dir = os.path.dirname(os.path.abspath("__file__"))
path = os.path.join(current_dir, '../../')
sys.path.append(path)
from dataset import plot_log, prepare_data, util
from transformer import get_train_valid_test
from model import define_model, multi_loss, my_callback, my_metric
from predicter import roc_curve, conf_matrix, multi_predict, grad_cam

import numpy as np
import keras
import keras.backend as K
import matplotlib.pyplot as plt

sys.path.append(r'/home/tmp10014/jupyterhub/notebook/other/lib_DL/mixup-generator')
from random_eraser import get_random_eraser

## define_model.get_fine_tuning_model
## define_model.get_optimizers
## my_metric.get_lr_metric
## multi_loss.build_masked_loss
## multi_loss.masked_accuracy

In [9]:
out_dir='output_test/331x331'
shape=[331, 331, 3]
num_classes=12
choice_model_list=['ResNet50', 'InceptionV3', 'NASNetLarge']# ['VGG16','ResNet50','InceptionV3','Xception','InceptionResNetV2','NASNetLarge','SEResNet154','SEInceptionV3','SEInceptionResNetV2']
FCnum_list=[None,1,4]# [None,1,2,3,4,5]
FCpool='GlobalAveragePooling2D'
Dence_1, Dropout_1, addBatchNorm_1, kernel_initializer_1, l2_rate_1=1024, 0.5, None, 'he_normal', 1e-4
Dence_2, Dropout_2, addBatchNorm_2, kernel_initializer_2, l2_rate_2=512, 0.5, None, 'he_normal', 1e-4
Dence_3, Dropout_3, addBatchNorm_3, kernel_initializer_3, l2_rate_3=256, 0.5, None, 'he_normal', 1e-4
Dence_4, Dropout_4, addBatchNorm_4, kernel_initializer_4, l2_rate_4=128, 0.5, None, 'he_normal', 1e-4
Dence_5, Dropout_5, addBatchNorm_5, kernel_initializer_5, l2_rate_5=64, 0.5, None, 'he_normal', 1e-4
pred_kernel_initializer='zeros'
pred_l2_rate=1e-4
activation_list=['sigmoid']# ['sigmoid','softmax']
trainable='all'
gpu_count=len(gpu_num.split(','))
skip_bn=True
opt_list=['sgd', 'adam', 'nadam']# ['sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam', 'adamax', 'nadam']

for choice_model in choice_model_list:
    for FCnum in FCnum_list:
        for activation in activation_list:
            print('\n###################################')
            model, orig_model = define_model.get_fine_tuning_model(out_dir, shape[0], shape[1], shape[2], num_classes
                                                                    , choice_model, trainable
                                                                    , FCnum
                                                                    , FCpool=FCpool
                                                                    , Dence_1=Dence_1, Dropout_1=Dropout_1, addBatchNorm_1=addBatchNorm_1, kernel_initializer_1=kernel_initializer_1, l2_rate_1=l2_rate_1
                                                                    , Dence_2=Dence_2, Dropout_2=Dropout_2, addBatchNorm_2=addBatchNorm_2, kernel_initializer_2=kernel_initializer_2, l2_rate_2=l2_rate_2
                                                                    , Dence_3=Dence_3, Dropout_3=Dropout_3, addBatchNorm_3=addBatchNorm_3, kernel_initializer_3=kernel_initializer_3, l2_rate_3=l2_rate_3
                                                                    , Dence_4=Dence_4, Dropout_4=Dropout_4, addBatchNorm_4=addBatchNorm_4, kernel_initializer_4=kernel_initializer_4, l2_rate_4=l2_rate_4
                                                                    , Dence_5=Dence_5, Dropout_5=Dropout_5, addBatchNorm_5=addBatchNorm_5, kernel_initializer_5=kernel_initializer_5, l2_rate_5=l2_rate_5
                                                                    , pred_kernel_initializer=pred_kernel_initializer, pred_l2_rate=pred_l2_rate
                                                                    , activation=activation 
                                                                    , gpu_count=gpu_count
                                                                    , skip_bn=skip_bn
                                                                  )
            for opt in opt_list:
                # オプティマイザ
                optim = define_model.get_optimizers(opt)
                lr_metric = my_metric.get_lr_metric(optim)
                # モデルコンパイル
                model.compile(loss=multi_loss.build_masked_loss(K.binary_crossentropy)
                              , optimizer=optim
                              , metrics=['accuracy', 'binary_accuracy', multi_loss.masked_accuracy, lr_metric])
model.summary()


###################################
----- model_param -----
output_dir = output_test/331x331
img_rows img_cols channels = 331 331 3
num_classes = 12
choice_model trainable = ResNet50 all
FCnum = None
FCpool = GlobalAveragePooling2D
pred_kernel_initializer pred_l2_rate = zeros 0.0001
activation = sigmoid
gpu_count = 1
skip_bn = True


KeyboardInterrupt: 

## model.fit_generator(No_branch)
### 331x331_SEResNet

In [4]:
%%time
out_dir='output_test/331x331'
num_classes=12
train_batch_size=4
valid_batch_size=6
shape=[331, 331, 3]
epochs=2
gpu_count=len(gpu_num.split(','))

# ラベル情報などが書いたcsvファイル（tox21_compoundData.csv）
csv_path = r'../../storage/tox21_compoundData.csv'
# IDにファイルのフルパスつける
img_dir = r'../../storage/img/Wash/100x100/'
# 画像ファイルのサフィックス
img_suffix = r'_000.jpg'
# 欠損値を-1に置換したtox21_compoundData.csvのデータフレーム
df = prepare_data.make_label_df(csv_path, img_dir, img_suffix=img_suffix)
# Tox21の画像のパスとラベル(y_train,y_valid,y_test)を取得する
train_files, validation_files, test_files, y_train, y_valid, y_test = prepare_data.make_train_val_test(df)

horizontal_flip=True
vertical_flip=True
rotation_range=60
zoom_range=[0.5, 1.9]
shear_range=0.2

d_cls = get_train_valid_test.LabeledDataset(shape, train_batch_size, valid_batch_size)
d_cls.load_train_as_image(train_files, y_train)
d_cls.load_validation_as_image(validation_files, y_valid)
d_cls.load_test_as_image(test_files, y_test)

# ImageDataGenerator のオプションを辞書型で詰める
# get_random_eraserでd_clsの値使うのでここで宣言しないとエラーになる
IDG_options={'horizontal_flip': horizontal_flip
             , 'vertical_flip': vertical_flip
             , 'rotation_range': rotation_range
             , 'zoom_range': zoom_range
             , 'shear_range': shear_range
             , 'preprocessing_function': get_random_eraser(v_l=np.min(d_cls.X_train), v_h=np.max(d_cls.X_train))
            }

d_cls.create_generator(IDG_options=IDG_options)

# 12の出力（task）をだすgeneratorにする
generator_multi_train = get_train_valid_test.generator_12output(d_cls.train_gen)
generator_multi_valid = get_train_valid_test.generator_12output(d_cls.valid_gen)

training_set_y.shape: (11764, 12)
validation_set_y.shape: (296, 12)
test_set_y.shape: (647, 12)
----- train_ImageDataGenerator -----
use_mixup: False
IDG_options: {'horizontal_flip': True, 'vertical_flip': True, 'rotation_range': 60, 'zoom_range': [0.5, 1.9], 'shear_range': 0.2, 'preprocessing_function': <function get_random_eraser.<locals>.eraser at 0x7f8c556bcbf8>}
CPU times: user 29.2 s, sys: 14.6 s, total: 43.8 s
Wall time: 1min 2s


In [5]:
%%time
epochs=1
choice_model_list=['SEResNet']# ['VGG16','ResNet50','InceptionV3','Xception','InceptionResNetV2','NASNetLarge','SEResNet154','SEInceptionV3','SEInceptionResNetV2']
seresnet_num_list=[101,154] # 18,34,50,
FCnum_list=[1]# [None,1,2,3,4,5]
FCpool='GlobalAveragePooling2D'
Dence_1, Dropout_1, addBatchNorm_1, kernel_initializer_1, l2_rate_1=1024, 0.5, None, 'he_normal', 1e-4
Dence_2, Dropout_2, addBatchNorm_2, kernel_initializer_2, l2_rate_2=512, 0.5, None, 'he_normal', 1e-4
Dence_3, Dropout_3, addBatchNorm_3, kernel_initializer_3, l2_rate_3=256, 0.5, None, 'he_normal', 1e-4
Dence_4, Dropout_4, addBatchNorm_4, kernel_initializer_4, l2_rate_4=128, 0.5, None, 'he_normal', 1e-4
Dence_5, Dropout_5, addBatchNorm_5, kernel_initializer_5, l2_rate_5=64, 0.5, None, 'he_normal', 1e-4
pred_kernel_initializer='zeros'
pred_l2_rate=1e-4
activation_list=['sigmoid']# ['sigmoid','softmax']
trainable='all'
skip_bn=True
opt_list=['sgd']# ['sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam', 'adamax', 'nadam']

for choice_model in choice_model_list:
    for seresnet_num in seresnet_num_list:
        for FCnum in FCnum_list:
            for activation in activation_list:
                print('\n###################################')
                model, orig_model = define_model.get_fine_tuning_model(out_dir, shape[0], shape[1], shape[2], num_classes
                                                                        , choice_model, trainable
                                                                        , FCnum
                                                                        , FCpool=FCpool
                                                                        , Dence_1=Dence_1, Dropout_1=Dropout_1, addBatchNorm_1=addBatchNorm_1, kernel_initializer_1=kernel_initializer_1, l2_rate_1=l2_rate_1
                                                                        , Dence_2=Dence_2, Dropout_2=Dropout_2, addBatchNorm_2=addBatchNorm_2, kernel_initializer_2=kernel_initializer_2, l2_rate_2=l2_rate_2
                                                                        , Dence_3=Dence_3, Dropout_3=Dropout_3, addBatchNorm_3=addBatchNorm_3, kernel_initializer_3=kernel_initializer_3, l2_rate_3=l2_rate_3
                                                                        , Dence_4=Dence_4, Dropout_4=Dropout_4, addBatchNorm_4=addBatchNorm_4, kernel_initializer_4=kernel_initializer_4, l2_rate_4=l2_rate_4
                                                                        , Dence_5=Dence_5, Dropout_5=Dropout_5, addBatchNorm_5=addBatchNorm_5, kernel_initializer_5=kernel_initializer_5, l2_rate_5=l2_rate_5
                                                                        , pred_kernel_initializer=pred_kernel_initializer, pred_l2_rate=pred_l2_rate
                                                                        , activation=activation 
                                                                        , gpu_count=gpu_count
                                                                        , skip_bn=skip_bn
                                                                        , seresnet_num=seresnet_num
                                                                      )
                for opt in opt_list:
                    # オプティマイザ
                    optim = define_model.get_optimizers(opt, decay=0.0)
                    lr_metric = my_metric.get_lr_metric(optim)# コールバックで下げたlrは表示されない。あくまでそのエポックの初期学習率をエポックごとに表示する
                    # モデルコンパイル
                    model.compile(loss=multi_loss.build_masked_loss(K.binary_crossentropy)
                                  , optimizer=optim
                                  , metrics=['accuracy', 'binary_accuracy', multi_loss.masked_accuracy, lr_metric])

                model.fit_generator(d_cls.train_gen
                        , steps_per_epoch=d_cls.train_steps_per_epoch()
                        , epochs=epochs
                        , validation_data=d_cls.valid_gen
                        , validation_steps=d_cls.valid_steps_per_epoch()
                        , verbose=1)
%matplotlib inline
plot_log.plot_results(out_dir, os.path.join(out_dir, 'history.tsv'))


###################################
----- model_param -----
output_dir = output_test
img_rows img_cols channels = 331 331 3
num_classes = 12
choice_model trainable = SEResNet all
FCnum = 1
FCpool = GlobalAveragePooling2D
pred_kernel_initializer pred_l2_rate = zeros 0.0001
activation = sigmoid
gpu_count = 1
skip_bn = True
seresnet_num = 101


/home/tmp10014/.conda/envs/py36/lib/python3.6/site-packages/keras/models.py:251: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


----- FC_layer -----
dence dropout addBatchNorm kernel_initializer l2_rate = 1024 0.5 None he_normal 0.0001
---- choice_optim = sgd ----
sgd_lr sgd_momentum sgd_decay sgd_nesterov = 0.01 0.9 0.0 True
Epoch 1/1
2941/2941 [==============================] - 2332s 793ms/step - loss: 0.3386 - acc: 0.1545 - binary_accuracy: 0.6461 - masked_accuracy: 0.9239 - lr: 0.0100 - val_loss: 0.3976 - val_acc: 0.0476 - val_binary_accuracy: 0.8078 - val_masked_accuracy: 0.9158 - val_lr: 0.0100

###################################
----- model_param -----
output_dir = output_test
img_rows img_cols channels = 331 331 3
num_classes = 12
choice_model trainable = SEResNet all
FCnum = 1
FCpool = GlobalAveragePooling2D
pred_kernel_initializer pred_l2_rate = zeros 0.0001
activation = sigmoid
gpu_count = 1
skip_bn = True
seresnet_num = 154
----- FC_layer -----
dence dropout addBatchNorm kernel_initializer l2_rate = 1024 0.5 None he_normal 0.0001
---- choice_optim = sgd ----
sgd_lr sgd_momentum sgd_decay sgd_nester

FileNotFoundError: File b'output_test/history.tsv' does not exist

## define_model.get_12branch_fine_tuning_model
- ニューラルネットワークを分岐させてMulti-task learningのときに使うもの

## define_model.get_optimizers
## my_metric.get_lr_metric
## multi_loss.build_masked_loss
## multi_loss.masked_accuracy

In [7]:
out_dir='output_test/331x331'
shape=[331, 331, 3]
num_classes=12
choice_model_list=['VGG16']# ['VGG16','ResNet50','InceptionV3','Xception','InceptionResNetV2','NASNetLarge','SEResNet154','SEInceptionV3','SEInceptionResNetV2']
FCnum_list=[None,1,2,3,4,5]
FCpool='GlobalAveragePooling2D'
Dence_1, Dropout_1, addBatchNorm_1, kernel_initializer_1, l2_rate_1=1024, 0.5, None, 'he_normal', 1e-4
Dence_2, Dropout_2, addBatchNorm_2, kernel_initializer_2, l2_rate_2=512, 0.5, None, 'he_normal', 1e-4
Dence_3, Dropout_3, addBatchNorm_3, kernel_initializer_3, l2_rate_3=256, 0.5, None, 'he_normal', 1e-4
Dence_4, Dropout_4, addBatchNorm_4, kernel_initializer_4, l2_rate_4=128, 0.5, None, 'he_normal', 1e-4
Dence_5, Dropout_5, addBatchNorm_5, kernel_initializer_5, l2_rate_5=64, 0.5, None, 'he_normal', 1e-4
pred_kernel_initializer='zeros'
pred_l2_rate=1e-4
activation_list=['sigmoid']# ['sigmoid','softmax']
trainable='all'
gpu_count=len(gpu_num.split(','))
skip_bn=True
opt_list=['sgd', 'adam', 'nadam']# ['sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam', 'adamax', 'nadam']

for choice_model in choice_model_list:
    for FCnum in FCnum_list:
        for activation in activation_list:
            print('\n###################################')
            model, orig_model = define_model.get_12branch_fine_tuning_model(out_dir, shape[0], shape[1], shape[2], num_classes
                                                                    , choice_model, trainable
                                                                    , FCnum
                                                                    , FCpool=FCpool
                                                                    , Dence_1=Dence_1, Dropout_1=Dropout_1, addBatchNorm_1=addBatchNorm_1, kernel_initializer_1=kernel_initializer_1, l2_rate_1=l2_rate_1
                                                                    , Dence_2=Dence_2, Dropout_2=Dropout_2, addBatchNorm_2=addBatchNorm_2, kernel_initializer_2=kernel_initializer_2, l2_rate_2=l2_rate_2
                                                                    , Dence_3=Dence_3, Dropout_3=Dropout_3, addBatchNorm_3=addBatchNorm_3, kernel_initializer_3=kernel_initializer_3, l2_rate_3=l2_rate_3
                                                                    , Dence_4=Dence_4, Dropout_4=Dropout_4, addBatchNorm_4=addBatchNorm_4, kernel_initializer_4=kernel_initializer_4, l2_rate_4=l2_rate_4
                                                                    , Dence_5=Dence_5, Dropout_5=Dropout_5, addBatchNorm_5=addBatchNorm_5, kernel_initializer_5=kernel_initializer_5, l2_rate_5=l2_rate_5
                                                                    , pred_kernel_initializer=pred_kernel_initializer, pred_l2_rate=pred_l2_rate
                                                                    , activation=activation 
                                                                    , gpu_count=gpu_count
                                                                    , skip_bn=skip_bn
                                                                  )
            for opt in opt_list:
                # オプティマイザ
                optim = define_model.get_optimizers(opt)
                lr_metric = my_metric.get_lr_metric(optim)
                # モデルコンパイル
                model.compile(loss=multi_loss.build_masked_loss(K.binary_crossentropy)
                              , optimizer=optim
                              , metrics=['accuracy', 'binary_accuracy', multi_loss.masked_accuracy, lr_metric])
model.summary()


###################################
----- model_param -----
output_dir = output_test
img_rows img_cols channels = 331 331 3
num_classes = 12
choice_model trainable = VGG16 all
FCnum = None
FCpool = GlobalAveragePooling2D
pred_kernel_initializer pred_l2_rate = zeros 0.0001
activation = sigmoid
gpu_count = 1
skip_bn = True


KeyboardInterrupt: 